In [1]:
# Bibliotecas
import sys
import os
import pandas as pd

# Bibliotecas
from datetime import datetime
import time
import schedule

In [2]:
# Variáveis
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("tests.ipynb"), "..", "env")))
from env_variables import apikey

In [3]:
# Scripts
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("tests.ipynb"), "..", "env")))
from criteria import Search_Criteria

sys.path.append(os.path.abspath(os.path.join(os.path.dirname("tests.ipynb"), "..", "api")))
from api import APIClient

sys.path.append(os.path.abspath(os.path.join(os.path.dirname("tests.ipynb"), "..", "pipeline")))
from pipeline import Data_Pipeline_API

# Testes

In [15]:
# Instanciando objeto de critérios de pesquisa
search_object = Search_Criteria("genomics", ["DNA", "genetic", "treatment"])

In [16]:
# Instanciando API
news_api = APIClient(apikey, search_object.subject)

# Adquirindo todos os artigos
news_api.get_articles()

In [17]:
# Instanciando Pipeline
pipeline_data = Data_Pipeline_API()

# Carregando dados da API
pipeline_data.load_api(news_api.articles)

# Salvando em Parquet
pipeline_data.save_data("raw_data")

In [18]:
# Importando DataFrame
pipeline_data.load_parquet("raw_data")

# Limpeza de dados
pipeline_data.data_cleaning()

# Salvando Parquet
pipeline_data.save_data("cleaned_data")

In [4]:
pipeline_data = Data_Pipeline_API()
pipeline_data.load_parquet("raw_data")
pipeline_data.df_parquet.shape

(577, 8)

In [5]:
pipeline_data.load_parquet("cleaned_data")
pipeline_data.df_parquet.shape

(577, 7)

# Funções de Rotina

In [ ]:
# Funçoes de rotina
def update_raw():
    now = datetime.now()

    # Adquirindo todos os artigos
    client.get_articles()

    # Instanciando Pipeline
    pipeline_data = Data_Pipeline_API()

    if len(client.articles) != 0:
        # Carregando dados da API
        pipeline_data.load_api(client.articles)

        # Salvando em Parquet - Dados brutos
        pipeline_data.save_data("raw_data")
        print(f"raw_data atualizado em {now.strftime('%Y-%m-%d %H:%M:%S')}")
    
    else:
        print(f"Sem atualizações em {now.strftime('%Y-%m-%d %H:%M:%S')}")
        time.sleep(30)


def update_cleaned():
    update_raw()
    now = datetime.now()

    # Instanciando Pipeline
    pipeline_data = Data_Pipeline_API()
    
    # Importando DataFrame
    pipeline_data.load_parquet("raw_data")

    # Limpeza de dados
    pipeline_data.data_cleaning()

    # Salvando Parquet
    pipeline_data.save_data("cleaned_data")
    print(f"cleaned_data atualizado em {now.strftime('%Y-%m-%d %H:%M:%S')}")
    time.sleep(780)

# Rotina

In [ ]:
# Instanciando pesquisa e palavras chave
criteria = Search_Criteria("genomics", ["DNA", "genetic", "treatment"])

# Instanciando API
client = APIClient(apikey, criteria.subject)

print("Script Iniciado")

schedule.every().hour.at(":00").do(update_cleaned)
schedule.every().hour.at(":15").do(update_raw)
schedule.every().hour.at(":30").do(update_raw)
schedule.every().hour.at(":45").do(update_raw)

while True:
    schedule.run_pending()
    time.sleep(20) 